In [7]:


import pandas as pd
import sys
from pathlib import Path
import pandas as pd


base_path = Path(r'C:\Users\w\Documents\my_csv_project\week2-data-work')


sys.path.append(str(base_path / "src"))

from bootcamp_data.etl import project_paths


file_path = project_paths.processed / "analytics_table.parquet"


df = pd.read_parquet(file_path)


print("rows:", len(df))
print("cols:", len(df.columns))
print(df.dtypes.head(10))
print(df.isna().sum().sort_values(ascending=False).head(10))

  processed data at  : C:\Users\w\Documents\my_csv_project\week2-data-work\data\processed
rows: 5
cols: 17
order_id               string[python]
user_id                string[python]
amount                        Float64
quantity                        Int64
created_at        datetime64[ns, UTC]
status                         object
status_clean                   object
amount__isna                     bool
quantity__isna                   bool
date                           object
dtype: object
amount           1
amount_winsor    1
quantity         1
created_at       1
hour             1
dow              1
month            1
year             1
date             1
order_id         0
dtype: int64


In [ ]:
import pandas as pd
from bootcamp_data.etl import project_paths


files = {
    "Analytics Table": "analytics_table.parquet",
    
}


for title, file_name in files.items():
    path = project_paths.processed / file_name
    
    if path.exists():
        df = pd.read_parquet(path)
        
        print(f"\n{'='*20} {title} {'='*20}")
        print(f" : {df.shape[0]}   {df.shape[1]} ")
        
       
        display(df.head(10)) 
    else:
        print(f" {file_name}    {path}")


==================== Analytics Table ====================
أبعاد الملف: 5 صف و 17 عمود


,order_id,user_id,amount,quantity,created_at,status,status_clean,amount__isna,quantity__isna,date,year,month,dow,hour,country,signup_date,amount_winsor
0,A0001,0001,12.5,1,2025-12-01 10:05:00+00:00,Paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,10.0,SA,11/15/2025,12.5
1,A0002,0002,8.0,2,2025-12-01 11:10:00+00:00,paid,paid,False,False,2025-12-01,2025.0,2025-12,Monday,11.0,SA,11/20/2025,8.135
2,A0003,0003,<NA>,1,2025-12-02 09:00:00+00:00,Refund,refund,True,False,2025-12-02,2025.0,2025-12,Tuesday,9.0,AE,11/22/2025,<NA>
3,A0004,0001,25.0,<NA>,2025-12-03 14:30:00+00:00,PAID,paid,False,True,2025-12-03,2025.0,2025-12,Wednesday,14.0,SA,11/15/2025,25.0
4,A0005,0004,100.0,1,NaT,paid,paid,False,False,None,NaN,<NA>,None,NaN,SA,11/25/2025,97.75


In [3]:
import sys 
!pip install seaborn matplotlib

In [17]:
import pandas as pd
import plotly.express as px


country_summary = df.groupby('country')['amount_winsor'].agg(
    n='count',
    mean='mean',
    median='median',
    p90=lambda x: x.quantile(0.9)
).round(2).reset_index()

country_summary.columns = ['group', 'n', 'mean', 'median', 'p90']

print("Summary Table: Purchasing Power Analysis")
display(country_summary)


fig = px.bar(country_summary, 
             x='group', 
             y=['mean', 'median'], 
             barmode='group',
             title='Average vs Median Purchasing Power (Checking for Skewness)',
             labels={'value': 'Amount', 'variable': 'Metric'})


csv_path = project_paths.root / "reports/country_analysis_table.csv"
country_summary.to_csv(csv_path, index=False)

image_path = project_paths.root / "reports/figures/revenue_by_country.png"

try:
    fig.write_image(str(image_path)) 
    print(f" Chart saved as IMAGE at: {image_path}")
except Exception as e:
    
    print("HTML")
    image_path = project_paths.root / "reports/figures/revenue_by_country.html"
    fig.write_html(str(image_path))

print(f" Report saved at: {csv_path}")
fig.show()

Summary Table: Purchasing Power Analysis


,group,n,mean,median,p90
0,AE,0,<NA>,<NA>,<NA>
1,SA,4,35.85,18.75,75.93


 Chart saved as IMAGE at: C:\Users\w\Documents\my_csv_project\week2-data-work\reports\figures\revenue_by_country.png
 Report saved at: C:\Users\w\Documents\my_csv_project\week2-data-work\reports\country_analysis_table.csv


In [19]:
### Question 2: How does the revenue trend evolve over time on a monthly basis?


import pandas as pd
import plotly.express as px
from pathlib import Path


trend = (
    df.groupby("month", dropna=False)
    .agg(
        n=("order_id", "size"),
        revenue=("amount_winsor", "sum"),
        mean=("amount_winsor", "mean"),   
        median=("amount_winsor", "median"), 
        p90=("amount_winsor", lambda x: x.quantile(0.9)) 
    )
    .reset_index()
    .sort_values("month")
)

print("Summary Table: Monthly Revenue Analysis")
display(trend.round(2))


fig = px.line(trend, x="month", y="revenue", 
              title="Monthly Revenue Trend (Audit Ready)",
              markers=True)


report_csv = project_paths.root / "reports/monthly_analysis_table.csv"
chart_image = project_paths.root / "reports/figures/revenue_trend_monthly.png"


trend.to_csv(report_csv, index=False)


try:
    fig.write_image(str(chart_image))
    print(f" Chart saved as PNG at: {chart_image}")
except Exception as e:
    
    chart_html = project_paths.root / "reports/figures/revenue_trend_monthly.html"
    fig.write_html(str(chart_html))
    print(f" Could not save PNG, saved as HTML instead: {chart_html}")

print(f" Table saved at: {report_csv}")
fig.show()

Summary Table: Monthly Revenue Analysis


,month,n,revenue,mean,median,p90
0,2025-12,4,45.64,15.21,12.5,22.5
1,<NA>,1,97.75,97.75,97.75,97.75


 Chart saved as PNG at: C:\Users\w\Documents\my_csv_project\week2-data-work\reports\figures\revenue_trend_monthly.png
 Table saved at: C:\Users\w\Documents\my_csv_project\week2-data-work\reports\monthly_analysis_table.csv


In [22]:
#Question 3 (Alternative): What is the relationship between "order status" and average transaction amounts?

#This question aims to examine whether higher transaction amounts tend to be more likely to fail or be refunded than lower amounts.


import pandas as pd
import plotly.express as px
from pathlib import Path


status_analysis = (
    df.groupby("status_clean", dropna=False)
    .agg(
        n=("order_id", "size"),
        total_revenue=("amount_winsor", "sum"),
        mean=("amount_winsor", "mean"),
        median=("amount_winsor", "median"),
        p90=("amount_winsor", lambda x: x.quantile(0.9))
    )
    .reset_index()
    .sort_values("mean", ascending=False)
)

print("Summary Table: Order Status vs. Transaction Amount")
display(status_analysis.round(2))


fig = px.bar(status_analysis, 
             x="status_clean", 
             y=["mean", "median"], 
             barmode="group",
             title="Average vs Median Amount by Order Status",
             labels={"value": "Amount", "variable": "Metric"})


report_csv = project_paths.root / "reports/status_analysis_table.csv"
chart_image = project_paths.root / "reports/figures/status_vs_amount.png"


status_analysis.to_csv(report_csv, index=False)

#
try:
    fig.write_image(str(chart_image))
    print(f" Chart saved as PNG at: {chart_image}")
except Exception as e:
    chart_html = project_paths.root / "reports/figures/status_vs_amount.html"
    fig.write_html(str(chart_html))
    print(f" Saved as HTML: {chart_html}")

fig.show()

Summary Table: Order Status vs. Transaction Amount


,status_clean,n,total_revenue,mean,median,p90
0,paid,4,143.38,35.85,18.75,75.93
1,refund,1,0.0,<NA>,<NA>,<NA>


 Chart saved as PNG at: C:\Users\w\Documents\my_csv_project\week2-data-work\reports\figures\status_vs_amount.png
